### Assignment 6: Caitlin Box

In [9]:
import pygame
import moderngl
import numpy
import glm
from LoadObject import getObjectData

In [10]:
pygame.init()

width = 840
height = 480
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Assignment 06: Mustaeen Ahmed")
gl = moderngl.get_context()
gl.info["GL_VERSION"]

'4.6.0 - Build 32.0.101.6881'

#### Get geometry data and load it to a GPU Buffer

In [11]:
geomData, bound = getObjectData("teapot_with_texCoords.obj", texture=True)

geomBuffer = gl.buffer(geomData)

Vertex position Count:  530 (5.929688, 4.125, 0.0)
Vertex normal Count:  530 (-0.966742, -0.255752, 0.0)
Vertex texture coordinate Count:  800 (2.0, 2.0, 0.0)
Face count:  992.0
texture exists


#### Vertex Shaders

In [12]:
face_vertex_shader_code = '''
#version 330 core

layout (location=0) in vec3 position;
layout (location=1) in vec2 uv;

out vec2 f_uv; // Texture coordinate
uniform mat4 view, perspective;

void main() {
    f_uv = uv;
    gl_Position = perspective*view*vec4(position, 1.0);
}
'''

face_fragment_shader_code = '''
#version 330 core
in vec2 f_uv;
uniform sampler2D map;
// Add output variable here
out vec4 color;

void main() {
    //set output color value here
    color = texture(map, f_uv);
}
'''

#### Programs and renderables

In [13]:
face_program = gl.program(
    vertex_shader= face_vertex_shader_code,
    fragment_shader= face_fragment_shader_code
)

face_renderable = gl.vertex_array(face_program,
    [(geomBuffer, "3f 2f", "position", "uv")]
)

#### Create texture object, specify filter and wrap option

In [14]:
texture_img = pygame.image.load("brick.jpg") 
texture_data = pygame.image.tobytes(texture_img,"RGB", True) 
faceTexture = gl.texture(texture_img.get_size(), data = texture_data, components=3)

faceTexture.build_mipmaps()

faceSampler = gl.sampler(texture=faceTexture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR)) 

#### Define Camera Parameters
the camera parameters for view transformation matrix computation are: eye_point, lookat_point/target_point, & up_vector

In [15]:
### Camera parameters
displacement = 2 * bound.radius * glm.rotate(glm.vec3(0,1,0), glm.radians(60), glm.vec3(1,0,0))
target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0,1,0)


### View volume parameters
fov = glm.radians(45)
aspect = width/height
near = bound.radius
far = 3 * bound.radius

perspectiveMatrix = glm.perspective(fov, width/height, near, far)

#### Render loop

In [16]:
running = True
clock = pygame.time.Clock()
alpha = 0
gl.enable(gl.DEPTH_TEST)

while running:   
    for event in pygame.event.get():
        # 27 is the esc key
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspectiveMatrix = glm.perspective(fov, width/height, near, far)

    # aspect ratio
    eye_point = glm.vec3(bound.center) + glm.rotate(displacement, glm.radians(alpha), glm.vec3(0,1,0))
    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)
    
    gl.clear(0.0,0.0,0.0) # background color
    
    face_program["view"].write(viewMatrix)
    face_program["perspective"].write(perspectiveMatrix)
    
    faceSampler.use(0)
    face_program["map"] = 0
    face_renderable.render(moderngl.TRIANGLES)
    
    pygame.display.flip()
    clock.tick(60)
    # make it move
    alpha = alpha + 1
    if alpha > 360:
        alpha = 0

pygame.display.quit()